## Import library

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
import pyvi

## Setup data

### Data path

In [2]:
ViCTSD_train = "..\\Dataset_Original\\ViCTSD\\ViCTSD_train.csv"
ViCTSD_test = "..\\Dataset_Original\\ViCTSD\\ViCTSD_test.csv"
vihsd_train = "..\\Dataset_Original\\vihsd\\train.csv"
vihsd_test = "..\\Dataset_Original\\vihsd\\test.csv"

### Visualize data

* Visualize record
* Unique labels each path

In [3]:
ViCTSD_train_data = pd.read_csv(ViCTSD_train)[['Comment', 'Toxicity']]
ViCTSD_test_data = pd.read_csv(ViCTSD_test)[['Comment', 'Toxicity']]
vihsd_train_data = pd.read_csv(vihsd_train)
vihsd_test_data = pd.read_csv(vihsd_test)

### Get text and label each DataFrame

In [4]:
vihsd_train_data = vihsd_train_data.rename(columns={'free_text': 'Comment', 'label_id': 'Toxicity'})
vihsd_test_data = vihsd_test_data.rename(columns={'free_text': 'Comment', 'label_id': 'Toxicity'})

In [5]:
vihsd_train_data['Toxicity'] = vihsd_train_data['Toxicity'].replace(2, 1)
vihsd_test_data['Toxicity'] = vihsd_test_data['Toxicity'].replace(2, 1)

### Merge DataFrame

In [6]:
df_train_joined = pd.concat([ViCTSD_train_data, vihsd_train_data], ignore_index=True)
df_test_joined = pd.concat([ViCTSD_test_data, vihsd_test_data], ignore_index=True)

In [7]:
# Setup type for data in Comment column
df_train_joined['Comment'] = df_train_joined['Comment'].astype(str)
df_test_joined['Comment'] = df_test_joined['Comment'].astype(str)

### Check null and remove null data

In [8]:
# Lấy các cột có giá trị null
def check_null_record(dataframe: pd.DataFrame, column_name: str):
    columns_with_nulls = dataframe[column_name].isnull()
    null_comment_indices = columns_with_nulls[columns_with_nulls == True].index
    # print null dataframe
    print(dataframe.iloc[null_comment_indices])
    return null_comment_indices

In [9]:
check_null_record(df_train_joined, 'Comment')

Empty DataFrame
Columns: [Comment, Toxicity]
Index: []


Index([], dtype='int64')

In [10]:
df_train_joined.drop(check_null_record(df_train_joined, 'Comment'), inplace=True)

Empty DataFrame
Columns: [Comment, Toxicity]
Index: []


### Preprocess data function
* Tokenize: tuyệt vời -> tuyệt_vời, ngu ngốc -> ngu_ngốc
* Loại stopwords: Thật tuyệt_vời -> tuyệt_vời
* Loại link, hashtag và tag trong comment

In [11]:
from pyvi import ViTokenizer, ViPosTagger
# word seqmentation
# ML không bắt buộc seqmentation
def tokenize(text):
    """
    Thật tuyệt vời -> Thật tuyệt_vời
    """
    return ViTokenizer.tokenize(text)

In [12]:
import os
with open('Stopwords\\vietnamese-stopwords-dash.txt', 'r', encoding='utf-8') as f:
    # Đọc từng dòng trong file
    stopwords = [line.strip() for line in f]
sus_stopwords = ["không","không_có","không_thể","chưa"]
for sus_stopword in sus_stopwords:
  stopwords.remove(sus_stopword)

In [13]:
def lower_and_remove_stopwords(text):
  try:
    # Chuyển đổi văn bản sang chữ thường
    text = text.lower()

    # Tách văn bản thành danh sách các từ
    words = text.split()

    # Loại bỏ stopword
    filtered_words = [word for word in words if word not in stopwords]

    # Ghép danh sách các từ đã lọc lại thành văn bản
    filtered_text = ' '.join(filtered_words)
  except Exception as e:
    print(text)
  return filtered_text


In [14]:
import re

def remove_links_hashtag_tag(text):
    """
    Removes URLs from a text string.

    Args:
        text: The text string to process.

    Returns:
        The text string with URLs removed.
    """
    link_remover = r"(https?://[^\s]+)"
    hashtag_remover = r"#[^\s]+"
    tag_remover = r"@[^\s]+"

    text = re.sub(link_remover, "", text)
    text = re.sub(hashtag_remover, "", text)
    text = re.sub(tag_remover, "", text)
  
    return text

In [15]:
# Calculate accuracy - out of 100 examples, what percentage does our model get right?
def accuracy_fn(y_true, y_pred):
  y_pred_rounded = torch.round(y_pred)  
  correct = torch.eq(y_true, y_pred_rounded).sum().item()
  try:
    acc = (correct/len(y_pred))*100
  except ZeroDivisionError:
    print("ZeroDivisionError")
    print(y_true, y_pred)
  return acc

#### Test một dữ liệu khi qua quá trình chuẩn bị

In [16]:
test_text = "bánh này không ngon"

In [17]:
test_text = ViTokenizer.tokenize(test_text)
test_text

'bánh này không ngon'

In [18]:
test_text = lower_and_remove_stopwords(test_text)
print(test_text)
test_text = remove_links_hashtag_tag(test_text)
test_text

bánh không ngon


'bánh không ngon'

In [19]:
text = "Đây là một ví dụ về #hashtag, http://www.example.com và @username."
result = remove_links_hashtag_tag(text)
result

'Đây là một ví dụ về   và '

## Create train, test

X_train, X_test, y_train, y_test

In [20]:
def preprocess_data(path1, path2):
    """
    1. Get data from 2 paths
    2. Concat 2 dataframes
    3. Tokenize -> lower and remove stopwords -> remove links, hashtags, tags
    4. Get X, y from dataframe (input, output)
    """
    train_data1 = pd.read_csv(path1)[['Comment', 'Toxicity']]
    train_data2 = pd.read_csv(path2)
    train_data2 = train_data2.rename(columns={'free_text': 'Comment', 'label_id': 'Toxicity'})
    train_data2['Toxicity'] = train_data2['Toxicity'].replace(2, 1)
    df_joined = pd.concat([train_data1, train_data2], ignore_index=True)
    df_joined['Comment'] = df_joined['Comment'].astype(str)
    df_joined['Comment'] = df_joined['Comment'].apply(tokenize)
    df_joined['Comment'] = df_joined['Comment'].apply(lower_and_remove_stopwords)
    df_joined['Comment'] = df_joined['Comment'].apply(remove_links_hashtag_tag)
    X = df_joined['Comment']
    y = df_joined['Toxicity'].tolist()
    return X, y, df_joined


In [21]:
import pandas as pd
X, y, train_data = preprocess_data(ViCTSD_train, vihsd_train)
X_test, y_test, test_data = preprocess_data(ViCTSD_test, vihsd_test)

In [22]:
train_data.to_csv('preprocessed_train_data.csv', index=False)
test_data.to_csv('preprocessed_test_data.csv', index=False)


### Load data from csv

In [23]:
import pandas as pd
train_data = pd.read_csv('preprocessed_train_data.csv')
test_data = pd.read_csv('preprocessed_test_data.csv')
X, y = train_data['Comment'].astype(str), train_data['Toxicity']

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab_size=50000
embedding_dim=32
max_length=140

# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded_sequences = pad_sequences(X_train_sequences, maxlen=max_length, padding='post',truncating='post')

# Thực hiện thay đổi test để đưa vào tính toán val_acc
X_val = tokenizer.texts_to_sequences(X_val)
X_val_padded = pad_sequences(X_val, maxlen=max_length, padding='post',truncating='post')

## Transforming data 

In [24]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class CustomDatasetv1(Dataset):
    def __init__(self, x_encoded: np.ndarray, y_encoded: pd.core.series.Series):
        # Setup
        self.x_encoded = x_encoded
        self.y_encoded = y_encoded.tolist()
    
    def __getitem__(self, idx):
        return (torch.FloatTensor(self.x_encoded[idx]), self.y_encoded[idx])
        # return (self.x_encoded[idx], self.y_encoded[idx])
        # return {'text': self.x[idx], 'label': self.y_encoded[idx]}
    
    def __len__(self):
        return self.x_encoded.shape[0]


In [25]:
train_data = CustomDatasetv1(X_train_padded_sequences, y_train)
test_data = CustomDatasetv1(X_val_padded, y_val)

In [26]:
len(train_data)

27943

In [27]:
from torch.utils.data import DataLoader
BATCH_SIZE=32
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)
test_dataloader = DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)                            

In [30]:
train_dataloader

In [36]:
from torch import nn

class ReshapeLayer(nn.Module):
    def forward(self, x):
        batch_size, sequence_length, embedding_dim = x.size()
        return x.view(batch_size, -1)

class model(nn.Module):
    def __init__(self, vocab_size:int, embedding_dim:int):
        super().__init__()
        self.se = nn.Sequential(
            nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim),
            ReshapeLayer(),  # Thêm Reshape Layer mới
            nn.Linear(in_features=140 * embedding_dim, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.se(x)

In [29]:
model = model(vocab_size=vocab_size, embedding_dim=embedding_dim)
model

model(
  (se): Sequential(
    (0): Embedding(50000, 32)
    (1): ReshapeLayer()
    (2): Linear(in_features=4480, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): ReLU()
    (6): Linear(in_features=20, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [30]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [32]:
y_pred_record = []

In [33]:
# Write a training and evaluationg loop for model_1
torch.manual_seed(42)

# import tqdm for progress bar
from tqdm.auto import tqdm

# Train for longer
epochs = 10

# # Put data on the target device
# X_padded_sequences, y_train = torch.tensor(X_padded_sequences).to(device), torch.tensor(y_train).to(device)
# padded_val_sequences, y_test=  torch.tensor(padded_val_sequences).to(device), torch.tensor(y_test).to(device)

# Create training and test loop
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n------")

  ### Training
  train_loss=0
  model.train()
  # Add a loop to loop through the training batches
  for batch, (X, y) in enumerate(train_dataloader):
    # print(f"Batch: {batch}")

    # 1. Forward
    X = X.long()
    y_pred = model(X)

    # 2. Calculate the loss
    y_pred_record.append(y_pred)
    loss = loss_fn(y_pred.squeeze(), y.float().squeeze())
    train_loss += loss

    # 3.
    optimizer.zero_grad()

    # 4.
    loss.backward()

    # 5.
    optimizer.step()

    # Print out what's happening
    # if batch %50==0:
    #   print(f"Looked at {batch*len(X)}/{len(train_dataloader.dataset)} samples.")

  # Divide total train loss by length of train dataloader
  train_loss /= len(train_dataloader)

  ### Testing
  test_loss, test_acc = 0,0
  model.eval()
  with torch.inference_mode():
    for X_test, y_test in test_dataloader:
      # 1. Forward pass
      X_test = X_test.long()
      test_pred = model(X_test)

      # 2. Calculate the loss (accumulatively)
      test_loss += loss_fn(test_pred.squeeze(dim=1), y_test.float())

      # 3. Calculate accuracy
      test_acc += accuracy_fn(y_true= y_test.float(),
                              y_pred = test_pred.squeeze(dim=1))

    # Calculate the test loss average per batch
    test_loss /= len(test_dataloader)

    # Calculate the test acc average per batch
    test_acc /= len(test_dataloader)

  # print out what happen
  print(f"\nTrain loss; {train_loss:.4f} | Test loss: {test_loss:.4f}, Test acc: {test_acc:.4f}")
  # print(f"\nTrain loss; {train_loss:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
------

Train loss; 0.4344 | Test loss: 0.4209, Test acc: 84.9490
Epoch: 1
------

Train loss; 0.3967 | Test loss: 0.3748, Test acc: 85.7781
Epoch: 2
------

Train loss; 0.3186 | Test loss: 0.4018, Test acc: 86.0969
Epoch: 3
------

Train loss; 0.2748 | Test loss: 0.3351, Test acc: 86.7347
Epoch: 4
------

Train loss; 0.2413 | Test loss: 0.3381, Test acc: 87.4681
Epoch: 5
------

Train loss; 0.2166 | Test loss: 0.3450, Test acc: 86.5434
Epoch: 6
------

Train loss; 0.1954 | Test loss: 0.3533, Test acc: 87.3087
Epoch: 7
------

Train loss; 0.1761 | Test loss: 0.3726, Test acc: 86.9579
Epoch: 8
------

Train loss; 0.1560 | Test loss: 0.4250, Test acc: 84.7577
Epoch: 9
------

Train loss; 0.1425 | Test loss: 0.4253, Test acc: 87.0855


## Lưu model

In [34]:
torch.save(model.state_dict(), 'my_model.pth')

## Load model

In [ ]:
from torch import nn

class ReshapeLayer(nn.Module):
    def forward(self, x):
        batch_size, sequence_length, embedding_dim = x.size()
        return x.view(batch_size, -1)

class model(nn.Module):
    def __init__(self, vocab_size:int, embedding_dim:int):
        super().__init__()
        self.se = nn.Sequential(
            nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim),
            ReshapeLayer(),  # Thêm Reshape Layer mới
            nn.Linear(in_features=140 * embedding_dim, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.se(x)

In [37]:
import torch
vocab_size=50000
embedding_dim=32
max_length=140
predict_model = model(vocab_size=vocab_size, embedding_dim=embedding_dim)
predict_model.load_state_dict(torch.load('my_model.pth'))

<All keys matched successfully>

## Dự đoán

### Nhập câu muốn dự đoán

In [38]:
test_sentences = [  "Đồ ngu, sao mày ngu thế?",
                    "Con này xấu xí, béo ú, chẳng ra gì cả.",
                    "Đồ đàn bà con gái chỉ biết ăn bám đàn ông.",
                    "Sản phẩm này tệ hại, ai mua cũng hối hận.",
                    "Mày hát dở tệ, đừng bao giờ hát nữa nhé.",
                    "Mày dám nói tao ngu à? Thử lặp lại lần nữa xem!",
                    "Đời mày xuống lỗ rồi con ạ",
                    "Ngu thì nín :))",
                    "khôn như vợ chồng nhà này k biết bao giờ mới giỗ đầu bạn nhỉ :))",
                    "toàn cái bọn lúc vay muốn dc vc.khi xong vc rồi thì vẩy đuôi b oi.giải thích bọn nó cố tính đéo hiểu đâu.còn vay gì trả đấy đơn giản chỉ thế.",
                    "sản phẩm này bố ỉa vào mà mua",
                    "bạn thật xinh đẹp",
                    "cộng hòa xã hội chủ nghĩa việt nam muôn năm",
                    "em thật dễ thương",
                    "hôm nay ăn gì",
                    "đầu bạn nhìn như đầu buổi",
                    "địt mẹ mày"
]

In [39]:
def predict(test_sentences):
    test_seq = tokenize(test_sentences)
    test_seq = lower_and_remove_stopwords(test_seq)
    test_seq = remove_links_hashtag_tag(test_seq)
    test_seq = tokenizer.texts_to_sequences([test_seq])
    # print(test_seq)
    test_seq = pad_sequences(test_seq, maxlen=max_length, padding='post',truncating='post')
    # print(test_seq)
    print(predict_model(torch.tensor(test_seq)))

In [40]:
for sentence in test_sentences:
    predict(sentence)

tensor([[0.9379]], grad_fn=<SigmoidBackward0>)
tensor([[0.4972]], grad_fn=<SigmoidBackward0>)
tensor([[0.4946]], grad_fn=<SigmoidBackward0>)
tensor([[0.1863]], grad_fn=<SigmoidBackward0>)
tensor([[0.9112]], grad_fn=<SigmoidBackward0>)
tensor([[0.4842]], grad_fn=<SigmoidBackward0>)
tensor([[0.2769]], grad_fn=<SigmoidBackward0>)
tensor([[0.9379]], grad_fn=<SigmoidBackward0>)
tensor([[0.0155]], grad_fn=<SigmoidBackward0>)
tensor([[0.1406]], grad_fn=<SigmoidBackward0>)
tensor([[0.0540]], grad_fn=<SigmoidBackward0>)
tensor([[0.0027]], grad_fn=<SigmoidBackward0>)
tensor([[0.3203]], grad_fn=<SigmoidBackward0>)
tensor([[0.0002]], grad_fn=<SigmoidBackward0>)
tensor([[0.0032]], grad_fn=<SigmoidBackward0>)
tensor([[0.0131]], grad_fn=<SigmoidBackward0>)
tensor([[0.9379]], grad_fn=<SigmoidBackward0>)


In [41]:
def predictV1(test_sentences):
    test_seq = tokenize(test_sentences)
    test_seq = lower_and_remove_stopwords(test_seq)
    test_seq = remove_links_hashtag_tag(test_seq)
    test_seq = tokenizer.texts_to_sequences([test_seq])
    # print(test_seq)
    test_seq = pad_sequences(test_seq, maxlen=max_length, padding='post',truncating='post')
    # print(test_seq)
    return predict_model(torch.tensor(test_seq))

In [42]:
X_test, y_test, test_data = preprocess_data(ViCTSD_test, vihsd_test)

In [43]:
num_record = len(X_test)
is_true = 0
true_0 = 0
false_0 = 0
true_1 = 0
false_1 = 0
test_predict = []
for i in range(len(X_test)):
    if predictV1(X_test[i])<0.5:
        test_predict.append(0)
        if y_test[i] == 0:
            true_0 += 1
            is_true += 1
        else:
            false_0 += 1
    else:
        test_predict.append(1)
        if y_test[i] == 1:
            true_1 += 1
            is_true += 1
        else:
            false_1 += 1
    

In [44]:
num_record, is_true, true_0, true_1,false_0 ,false_1

(7680, 6670, 6103, 567, 675, 335)

In [45]:
torch.eq(torch.tensor(test_predict), torch.tensor(y_test)).sum().item()

6670

In [47]:
# Calculate Accuracy:
torch.eq(torch.tensor(test_predict), torch.tensor(y_test)).sum().item()/num_record

0.8684895833333334

In [46]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
print(classification_report(y_true=y_test, y_pred=test_predict))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92      6438
           1       0.63      0.46      0.53      1242

    accuracy                           0.87      7680
   macro avg       0.76      0.70      0.73      7680
weighted avg       0.86      0.87      0.86      7680

